In [ ]:
!pip install pandas
import pandas as pd
df = pd.read_csv("./data.csv", parse_dates=[["Date","Time"]])
df_glucose_sensor = df.iloc[2689:21338]
df_glucose_sensor.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]

In [ ]:
!pip install pandas
import pandas as pd
df = pd.read_csv("./data.csv", parse_dates=[["Date","Time"]])
df_daily_basal_rate = df.iloc[2620:2684]
df_daily_basal_rate.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]

In [ ]:
# Install the tsfm library
! pip install "tsfm_public[notebooks] @ git+https://github.com/ibm-granite/granite-tsfm.git@v0.2.17" -U
import matplotlib.pyplot as plt
import pandas as pd

from tsfm_public import (
    TimeSeriesForecastingPipeline,
    TinyTimeMixerForPrediction,
)
from tsfm_public.toolkit.visualization import plot_predictions
timestamp_column = "Date_Time"
target_columns = ["Sensor Glucose (mg/dL)"]
context_length = 1536
input_df = pd.read_csv("./data.csv", parse_dates=[["Date","Time"]])
input_df = df.iloc[2689:21338]
input_df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]
fig, axs = plt.subplots(len(target_columns), 1, figsize=(10, 2 * len(target_columns)), squeeze=False)
for ax, target_column in zip(axs, target_columns):
    ax[0].plot(input_df[timestamp_column], input_df[target_column])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from tsfm_public import (
    TimeSeriesForecastingPipeline,
    TinyTimeMixerForPrediction,
)
from tsfm_public.toolkit.visualization import plot_predictions
timestamp_column = "Date_Time"
target_columns = ["Sensor Glucose (mg/dL)"]
context_length = 512
input_df = pd.read_csv("./data.csv", parse_dates=[["Date","Time"]])
input_df = df.iloc[2689:21337]
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])
input_df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]


In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
# input_df = input_df.iloc[2689:21338,:]
input_df = input_df.iloc[2689:5000,:]


# Fill NA/NaN values by propagating the last valid value.
input_df = input_df.ffill()

# Only use the last `context_length` rows for prediction.
# input_df = input_df.iloc[-context_length:,]
input_df = input_df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]
len(input_df)
input_df.dropna()

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])
timestamp_column = ["Date_Time"]
target_column = ["Sensor Glucose (mg/dL)"]
input_df = input_df.sort_values("Date_Time")
#input_df = pd.to_numeric(input_df["Sensor Glucose (mg/dL)"]).fillna(0).astype(int)
input_df.plot(x="Date_Time", y="Sensor Glucose (mg/dL)", figsize=(20, 10))

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)

context_length = 1024

input_df = input_df.iloc[2689:5000,:]

# Fill NA/NaN values by propagating the last valid value.
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Sensor Glucose (mg/dL)"]]
input_df.dropna()
input_df = input_df.iloc[-context_length:,]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

timestamp_column = ["Date_Time"]
target_column = ["Sensor Glucose (mg/dL)"]

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="Sensor Glucose (mg/dL)", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="5m",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="5m",
    timestamp_column="Date_Time",
    channel="Sensor Glucose (mg/dL)",
    indices=[-1],
    num_plots=1,
)

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
input_df = input_df.iloc[2:2615,:]

# Fill NA/NaN values by propagating the last valid value.
#input_df = input_df.fillna(0)
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]
#input_df.dropna()

timestamp_column = ["Date_Time"]
input_df["smooth_bolus"] = input_df["Bolus Volume Delivered (U)"].rolling(window=3).mean()
target_column = ["smooth_bolus"]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="smooth_bolus", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="5m",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="h",
    timestamp_column="Date_Time",
    channel="smooth_bolus",
    indices=[-1],
    num_plots=1,
)

In [ ]:
DATA_FILE_PATH = "data.csv"
input_df = pd.read_csv(
    DATA_FILE_PATH,
    parse_dates=[["Date", "Time"]],  # Parse the timestamp values as dates.
)
input_df = input_df.iloc[2620:2684,:]

# Fill NA/NaN values by propagating the last valid value.
#input_df = input_df.fillna(0)
input_df = input_df.ffill()

input_df = input_df.loc[:,["Date_Time","Bolus Volume Delivered (U)"]]
#input_df.dropna()

timestamp_column = ["Date_Time"]
target_column = ["Bolus Volume Delivered (U)"]

input_df[target_column] = input_df[target_column].astype(float)
input_df["Date_Time"] = pd.to_datetime(input_df["Date_Time"])

input_df = input_df.sort_values("Date_Time")
input_df.plot(x="Date_Time", y="Bolus Volume Delivered (U)", figsize=(20, 10))

zeroshot_model = TinyTimeMixerForPrediction.from_pretrained(
    "ibm-granite/granite-timeseries-ttm-r2",  # Name of the model on Hugging Face
    num_input_channels=len(target_column),  # tsp.num_input_channels
)

# Create a pipeline.
pipeline = TimeSeriesForecastingPipeline(
    zeroshot_model,
    timestamp_column="Date_Time",
    id_columns=[],
    target_columns=target_column,
    explode_forecasts=False,
    freq="d",
    device="cpu",  # Specify your local GPU or CPU.
)

# Make a forecast on the target column given the input data.
zeroshot_forecast = pipeline(input_df)

# Plot the historical data and predicted series.
plot_predictions(
    input_df=input_df,
    predictions_df=zeroshot_forecast,
    freq="d",
    timestamp_column="Date_Time",
    channel="Bolus Volume Delivered (U)",
    indices=[-1],
    num_plots=1,
)